In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.train.sample.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'textattack/bert-base-uncased-imdb',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [7]:
from typing import Generator
import torch
import numpy as np
import pandas as pd
from scipy.spatial import distance


def analysis(
        data: pd.DataFrame,
        dim: list = None
) -> Generator:
    if dim is None:
        dim = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]

    embed_col: np.ndarray = torch.stack(data[enc.col_name].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == enc.dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        # compute centroid means and dispersion for each cluster
        for label, group in data.groupby(dataset.target_label):
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

        record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

        yield record

In [8]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.936666e+06,1.245197e+06,7.870501
1,576,1.936999e+06,1.245412e+06,7.871798
2,384,1.936758e+06,1.245218e+06,7.873773
3,192,1.935962e+06,1.244567e+06,7.879527
4,96,1.934210e+06,1.243108e+06,7.893193
5,48,1.930430e+06,1.239993e+06,7.921449
6,24,1.922074e+06,1.233175e+06,7.983199
7,12,1.902749e+06,1.217704e+06,8.119086
8,6,1.854138e+06,1.178891e+06,8.451413
9,3,1.711572e+06,1.075962e+06,9.236423


In [9]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.936666e+06,1.245197e+06,7.870501
1,576,1.936996e+06,1.245416e+06,7.871884
2,384,1.936735e+06,1.245200e+06,7.874029
3,192,1.935962e+06,1.244544e+06,7.880069
4,96,1.934237e+06,1.243128e+06,7.892990
5,48,1.930401e+06,1.239985e+06,7.921559
6,24,1.921991e+06,1.233214e+06,7.983278
7,12,1.903017e+06,1.217892e+06,8.116464
8,6,1.853819e+06,1.179741e+06,8.439162
9,3,1.710758e+06,1.075000e+06,9.243265


In [10]:
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']].to_csv('./cluster_analysis.csv')